In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder

## Input data

In [2]:
df = pd.read_csv('./Churn_Modelling.csv')

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [5]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
X, y = df.drop(['Exited'], axis=1), df['Exited']

## Preprocessing data

In [7]:
geography_encoder = LabelEncoder()
gender_encoder = LabelEncoder()

In [8]:
X['Geography'] = geography_encoder.fit_transform(X['Geography'])

In [9]:
X['Gender'] = gender_encoder.fit_transform(X['Gender'])

In [10]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,0,42,2,0.00,1,1,1,101348.88
1,608,2,0,41,1,83807.86,1,0,1,112542.58
2,502,0,0,42,8,159660.80,3,1,0,113931.57
3,699,0,0,39,1,0.00,2,0,0,93826.63
4,850,2,0,43,2,125510.82,1,1,1,79084.10


In [11]:
X['Geography'].unique()

array([0, 2, 1])

In [12]:
one_hot_encoder = OneHotEncoder(categorical_features=[1])

In [13]:
X_encoded = one_hot_encoder.fit_transform(X).toarray()

In [14]:
X_encoded[0]

array([1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 6.1900000e+02,
       0.0000000e+00, 4.2000000e+01, 2.0000000e+00, 0.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0134888e+05])

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.3, random_state=101)

In [16]:
scaler = MinMaxScaler()

In [17]:
scaled_X_train = scaler.fit_transform(X_train)

In [18]:
print(X_train[0])
print(scaled_X_train[0])

[0.0000000e+00 0.0000000e+00 1.0000000e+00 5.1100000e+02 0.0000000e+00
 2.9000000e+01 9.0000000e+00 0.0000000e+00 2.0000000e+00 0.0000000e+00
 1.0000000e+00 1.4067698e+05]
[0.         0.         1.         0.322      0.         0.14864865
 0.9        0.         0.33333333 0.         1.         0.70347065]


In [19]:
scaled_X_test = scaler.transform(X_test)

In [20]:
scaled_X_test[0]

array([0.        , 0.        , 1.        , 0.538     , 1.        ,
       0.45945946, 0.8       , 0.        , 0.33333333, 1.        ,
       1.        , 0.61627849])

## Model

In [21]:
def get_classifier(optimizer='adam'):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(24, activation='relu', input_dim=12))
    model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.Dense(12, activation='relu'))
    model.add(tf.keras.layers.Dense(2, activation='softmax'))
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [22]:
model = get_classifier()

In [23]:
model.fit(scaled_X_train, y_train, epochs=25)

Epoch 1/25
7000/7000 [==============================]7000/7000 [==============================] - 1s 94us/step - loss: 0.5052 - acc: 0.7886

Epoch 2/25
7000/7000 [==============================]7000/7000 [==============================] - 0s 57us/step - loss: 0.4699 - acc: 0.7991

Epoch 3/25
7000/7000 [==============================]7000/7000 [==============================] - 0s 63us/step - loss: 0.4552 - acc: 0.8027

Epoch 4/25
7000/7000 [==============================]7000/7000 [==============================] - 0s 54us/step - loss: 0.4464 - acc: 0.8080

Epoch 5/25
7000/7000 [==============================]7000/7000 [==============================] - 0s 51us/step - loss: 0.4408 - acc: 0.8109

Epoch 6/25
7000/7000 [==============================]7000/7000 [==============================] - 0s 56us/step - loss: 0.4320 - acc: 0.8173

Epoch 7/25
7000/7000 [==============================]7000/7000 [==============================] - 0s 49us/step - loss: 0.4209 - acc: 0.8234

Epoch 8/25
70

## Prediction

In [24]:
y_pred = model.predict(scaled_X_test)

In [25]:
y_pred

array([[0.9707305 , 0.02926952],
       [0.96448994, 0.03551003],
       [0.7127436 , 0.28725642],
       ...,
       [0.9788674 , 0.02113254],
       [0.4786541 , 0.521346  ],
       [0.6893471 , 0.31065288]], dtype=float32)

In [26]:
from sklearn.metrics import classification_report

In [27]:
y_pred = [np.argmax(x) for x in y_pred]

In [28]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.87      0.97      0.92      2378
          1       0.81      0.43      0.56       622

avg / total       0.86      0.86      0.84      3000



So should we say goodbye to that customer ?

```
Geography: France
Credit Score: 600
Gender: Male
Age: 40 years old
Tenure: 3 years
Balance: $60000
Number of Products: 2
Does this customer have a credit card ? Yes
Is this customer an Active Member: Yes
Estimated Salary: $50000
```

In [29]:
geography_encoder.transform(['France'])

array([0])

In [30]:
gender_encoder.transform(['Male'])

array([1])

In [31]:
print(X.iloc[0])
print(X.iloc[0].values)

CreditScore           619.00
Geography               0.00
Gender                  0.00
Age                    42.00
Tenure                  2.00
Balance                 0.00
NumOfProducts           1.00
HasCrCard               1.00
IsActiveMember          1.00
EstimatedSalary    101348.88
Name: 0, dtype: float64
[6.1900000e+02 0.0000000e+00 0.0000000e+00 4.2000000e+01 2.0000000e+00
 0.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0134888e+05]


In [32]:
new_data = np.array([
    600,
    geography_encoder.transform(['France']),
    gender_encoder.transform(['Male']),
    40,
    3,
    60000,
    2,
    1,
    1,
    50000
])

In [33]:
new_data = one_hot_encoder.transform([new_data]).toarray()

In [34]:
print(new_data)

[[1.e+00 0.e+00 0.e+00 6.e+02 1.e+00 4.e+01 3.e+00 6.e+04 2.e+00 1.e+00
  1.e+00 5.e+04]]


In [35]:
model.predict(new_data)

array([[0., 1.]], dtype=float32)

In [36]:
y_pred = [np.argmax(x) for x in model.predict(new_data)]

In [37]:
y_pred

[1]

## k-fold cross validation

In [38]:
model = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=get_classifier, batch_size=12, epochs=10)

In [39]:
accuracies = cross_val_score(estimator=model, X=scaled_X_train, y=y_train, cv=2)

Epoch 1/10
3500/3500 [==============================]3500/3500 [==============================] - 1s 214us/step - loss: 0.5107 - acc: 0.7751

Epoch 2/10
3500/3500 [==============================]3500/3500 [==============================] - 1s 146us/step - loss: 0.4709 - acc: 0.7969

Epoch 3/10
3500/3500 [==============================]3500/3500 [==============================] - 0s 137us/step - loss: 0.4615 - acc: 0.7991

Epoch 4/10
3500/3500 [==============================]3500/3500 [==============================] - 0s 133us/step - loss: 0.4508 - acc: 0.8086

Epoch 5/10
3500/3500 [==============================]3500/3500 [==============================] - 0s 135us/step - loss: 0.4427 - acc: 0.8103

Epoch 6/10
3500/3500 [==============================]3500/3500 [==============================] - 0s 132us/step - loss: 0.4330 - acc: 0.8134

Epoch 7/10
3500/3500 [==============================]3500/3500 [==============================] - 0s 138us/step - loss: 0.4255 - acc: 0.8183

Epoch 

In [40]:
accuracies.mean()

0.8331428575686046

In [41]:
accuracies.std()

0.005142857364245801

## Grid search for best hyperparameters

In [42]:
parameters = {
    'batch_size': [16, 32],
    'epochs': [5, 10],
    'optimizer': ['adam', 'rmsprop']
}

In [43]:
model = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=get_classifier)

In [44]:
grid_search = GridSearchCV(estimator=model, param_grid=parameters, scoring='accuracy', cv=2)

In [45]:
result = grid_search.fit(X=scaled_X_train, y=y_train)

Epoch 1/5
3500/3500 [==============================]3500/3500 [==============================] - 1s 205us/step - loss: 0.4973 - acc: 0.7929

Epoch 2/5
3500/3500 [==============================]3500/3500 [==============================] - 0s 105us/step - loss: 0.4753 - acc: 0.7951

Epoch 3/5
3500/3500 [==============================]3500/3500 [==============================] - 0s 106us/step - loss: 0.4643 - acc: 0.8011

Epoch 4/5
3500/3500 [==============================]3500/3500 [==============================] - 0s 114us/step - loss: 0.4439 - acc: 0.8080

Epoch 5/5
3500/3500 [==============================]3500/3500 [==============================] - 0s 109us/step - loss: 0.4362 - acc: 0.8166

Epoch 1/5
3500/3500 [==============================]3500/3500 [==============================] - 1s 213us/step - loss: 0.5151 - acc: 0.7829

Epoch 2/5
3500/3500 [==============================]3500/3500 [==============================] - 0s 107us/step - loss: 0.4703 - acc: 0.8014

Epoch 3/5
350

In [46]:
result.best_params_

{'batch_size': 16, 'epochs': 10, 'optimizer': 'adam'}

In [47]:
result.best_score_

0.8328571428571429